In [ ]:
import os
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts

from inter_view.dashboards import SegmentationDashBoard, DashBoardCallback

# config

In [ ]:
outdir = '../../data/3D_corrections'
basedir = '../../data/3D'
data_pattern = '{subdir}/{fname}-T{time:04d}.{ext}'
index = ['subdir', 'time']

channel_config = {'membrane':{'cmap':'red'},
                  'nuclei':{'cmap':'gray'},
                  'nuclei_seg':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.5}}


opts.defaults(opts.Image('channel.nuclei', frame_width=1000),
              opts.Image('channel.nuclei_seg', frame_width=1000, clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1), tools=['hover'], show_title=False),)

# parse files

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[['membrane', 'nuclei', 'nuclei_seg']]

# filter samples for which the segmentation is available
df = df.groupby(['time'],).filter(lambda x: 'nuclei_seg' in x.reset_index().subdir.tolist())


df

# interactive dashboard
Arbitrary callbacks acting on the currently open image can be setup. The example below illustrate how to copy the current nuclei image and its segmentation in a "correction" folder

In [ ]:
import panel as pn
import param
    
class SegmentationCallbackDashBoard(SegmentationDashBoard, DashBoardCallback):
    ''''''

    @param.depends('_complete_update_counter')
    def widgets(self):
        wg = [super().widgets(), self._export_widgets()]
        return pn.Column(*wg)

In [ ]:
import shutil
import numpy as np

def copy_for_training(db):    
    subdf = db.subdf.dc[['nuclei', 'nuclei_seg']].reset_index()
    
    subdf_out = subdf.copy()
    subdf_out.basedir = db.out_folder
    subdf_out.replace('nuclei_seg', 'nuclei_annot', inplace=True)
    
    subdf.set_index('subdir', inplace=True)
    subdf_out.set_index('subdir', inplace=True)
    
    # copy channel
    ch_in_path = subdf.dc['nuclei'].dc.path[0]
    ch_out_path = subdf_out.dc['nuclei'].dc.path[0]
    os.makedirs(os.path.dirname(ch_out_path), exist_ok=True)
    shutil.copyfile(ch_in_path, ch_out_path)
    
    # save labels as signed int 16
    pred = subdf.dc['nuclei_seg'].dc.read()[0]
    subdf_out.dc['nuclei_annot'].dc.write(pred.astype(np.int16), compress=9)

    
copy_for_training.name = 'copy nuclei seg for correction'

db = SegmentationCallbackDashBoard(df=df,
                                   multi_select_levels=['subdir'],
                                   channel_config=channel_config,
                                   composite_channels=['membrane', 'nuclei'],
                                   overlay_channels=['nuclei_seg'],
                                   export_funs=[copy_for_training],
                                   out_folder=outdir
                                  )

db.panel().servable()